In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:

data = pd.read_csv('../data/combined_keypoints.csv')

print("First few rows of the dataset:")
print(data.head())

print(f"Dataset shape: {data.shape}")

label_encoder = LabelEncoder()
data['Class'] = label_encoder.fit_transform(data['Class'])
print("\nMapping of classes:")
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

X = data.drop(columns=['Class'])
y = data['Class']

num_classes = len(label_encoder.classes_)
y = to_categorical(y, num_classes=num_classes)

First few rows of the dataset:
        Class    Nose_x    Nose_y  Nose_conf  LeftEye_x  LeftEye_y  \
0  Warrior_II  0.275027  0.441496   0.838373   0.257330   0.454769   
1  Warrior_II  0.361404  0.488448   0.632348   0.354708   0.496576   
2  Warrior_II  0.226947  0.543324   0.787434   0.213398   0.533082   
3  Warrior_II  0.392478  0.374899   0.328839   0.381358   0.370187   
4  Warrior_II  0.223151  0.476531   0.887696   0.211527   0.493164   

   LeftEye_conf  RightEye_x  RightEye_y  RightEye_conf  ...  LeftKnee_conf  \
0      0.874609    0.258129    0.449623       0.896995  ...       0.941765   
1      0.715429    0.351345    0.491862       0.855811  ...       0.829439   
2      0.857410    0.214061    0.525327       0.864843  ...       0.923154   
3      0.546548    0.383478    0.369107       0.577419  ...       0.324949   
4      0.910345    0.209859    0.482766       0.860895  ...       0.828863   

   RightKnee_x  RightKnee_y  RightKnee_conf  LeftAnkle_x  LeftAnkle_y  \
0     

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

print("\nShape of X_train after reshaping:", X_train.shape)
print("Shape of X_test after reshaping:", X_test.shape)


Shape of X_train after reshaping: (10981, 1, 51)
Shape of X_test after reshaping: (2746, 1, 51)


In [ ]:
def create_model():
    model = Sequential()

    model.add(Bidirectional(LSTM(units=512, return_sequences=True, kernel_regularizer=l2(0.01)), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(units=256, return_sequences=True, kernel_regularizer=l2(0.01))))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(units=128, kernel_regularizer=l2(0.01))))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(units=256, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(units=128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Dense(units=64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.1))

    model.add(Dense(units=num_classes, activation='softmax'))

    optimizer = 'adam'
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-6, verbose=1)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1

for train_index, val_index in kf.split(X_train):
    print(f"\nTraining fold {fold_no}...")

    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    model = create_model()

    history = model.fit(
        X_train_fold, y_train_fold,
        epochs=150,
        batch_size=64,
        validation_data=(X_val_fold, y_val_fold),
        callbacks=[reduce_lr],
        verbose=1
    )

    fold_no += 1


Training fold 1...


2024-08-12 21:47:50.321843: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-08-12 21:47:52.843901: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-12 21:47:52.848265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-12 21:47:52.8

Epoch 1/150


2024-08-12 21:47:59.051636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-12 21:47:59.059942: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-12 21:47:59.067354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

138/138 [==============================] - ETA: 0s - loss: 20.1806 - accuracy: 0.1765

...

Epoch 1/150


2024-08-13 03:14:35.405486: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-13 03:14:35.409726: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-13 03:14:35.413398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

138/138 [==============================] - ETA: 0s - loss: 19.9943 - accuracy: 0.1789

2024-08-13 03:15:26.848016: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-13 03:15:26.852402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-13 03:15:26.856180: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

138/138 [==============================] - 58s 241ms/step - loss: 19.9943 - accuracy: 0.1789 - val_loss: 11.8743 - val_accuracy: 0.0128 - lr: 0.0010
Epoch 2/150
138/138 [==============================] - 31s 222ms/step - loss: 7.1465 - accuracy: 0.4064 - val_loss: 7.5952 - val_accuracy: 0.0141 - lr: 0.0010
Epoch 3/150
138/138 [==============================] - 30s 215ms/step - loss: 4.2240 - accuracy: 0.4776 - val_loss: 6.0729 - val_accuracy: 0.0137 - lr: 0.0010
Epoch 4/150
138/138 [==============================] - 30s 219ms/step - loss: 3.2419 - accuracy: 0.5224 - val_loss: 4.3371 - val_accuracy: 0.1890 - lr: 0.0010
Epoch 5/150
138/138 [==============================] - 30s 220ms/step - loss: 2.8039 - accuracy: 0.5521 - val_loss: 3.6055 - val_accuracy: 0.3092 - lr: 0.0010
Epoch 6/150
138/138 [==============================] - 31s 222ms/step - loss: 2.6263 - accuracy: 0.5638 - val_loss: 2.7103 - val_accuracy: 0.5387 - lr: 0.0010
Epoch 7/150
138/138 [==============================] - 2

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
print(f"Validation accuracy for fold {fold_no - 1}: {val_accuracy * 100:.2f}%")

Validation accuracy for fold 5: 79.10%


In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
test_accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f'\nFinal test accuracy: {test_accuracy * 100:.2f}%')

2024-08-13 04:42:21.897945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-13 04:42:21.901861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-13 04:42:21.905123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

86/86 [==============================] - 8s 34ms/step

Final test accuracy: 79.24%


In [ ]:
final_loss, final_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'\nFinal test accuracy: {final_accuracy * 100:.2f}%')


Final test accuracy: 79.24%


In [ ]:
report = classification_report(y_test_classes, y_pred_classes)
print(f"Classification Report \n{report}")

Classification Report 
              precision    recall  f1-score   support

           0       0.83      0.91      0.87        22
           1       1.00      0.86      0.92        49
           2       0.87      0.70      0.78        37
           3       0.86      0.84      0.85        38
           4       0.93      0.84      0.89        32
           5       0.87      0.96      0.92        28
           6       0.81      0.97      0.88        39
           7       0.88      0.82      0.85        28
           8       0.83      0.97      0.89        39
           9       0.93      0.91      0.92        97
          10       0.89      0.70      0.78        23
          11       0.73      0.44      0.55        36
          12       0.63      0.91      0.74        32
          13       0.81      0.88      0.84        48
          14       0.94      0.85      0.89        20
          15       1.00      0.35      0.52        23
          16       0.97      0.91      0.94        33
    